# Процедура дискриминантного анализа

In [3]:
# Здесь подгрузим необходимые библиотеки для проведения процедуры дискриминационного анализа

import numpy as np
import pandas as pd
from IPython.display import HTML

## Исходные данные

In [4]:
'''
Здесь будем подгружать их. Для начала подгрузим группу обучющую группу М1 из примера. Столбцы - предприятия, строки - факторы
В реальной работе, сюда нужно будет загрузить таблицу уровня жизни центрального федерального округа.
Столбцы - регионы, строки - факторы.
'''
M1 = pd.read_csv('M1.csv')

In [5]:
HTML(M1.to_html(index=False))

№ предприятия в группе,X1,X2,X3
1,224.228,17.115,22.981
2,151.827,14.904,21.481
3,147.313,13.627,28.669
4,152.253,10.545,10.199


In [6]:
'''
То же самое сделаем с группой М2 из примера. 
В реальной работе, здесь будет таблица уровня жизни приволжского федерального оеруга
'''
M2 = pd.read_csv('M2.csv')
HTML(M2.to_html(index=False))

№ предприятия в группе,X1,X2,X3
1,46.757,4.428,11.124
2,29.033,5.510,6.091
3,52.134,4.214,11.842
4,37.050,5.527,11.873
5,63.979,4.211,12.860


In [7]:
'''
Сюда загружаем то, что нам требуется классифицировать М0- из примера. В реальной работе,
здесь будет таблиц уровня жизни северозападного федерального округа
'''
M0 = pd.read_csv('M0.csv')
HTML(M0.to_html(index=False))

№ предприятия в группе,X1,X2,X3
1,55.451,9.592,12.840
2,78.575,11.727,15.535
3,98.353,17.572,20.458


## Получение обучающих подмножеств

### Получим вектора средних значений признаков по обучающим группам М1 и М2 

In [8]:
def calcMean(X):
    mean = np.mean(X, axis = 0)
    return np.reshape(np.array(mean), (-1,1))

M1,M2 = M1.iloc[:,1:].to_numpy(), M2.iloc[:,1:].to_numpy()
    
X1,X2 = calcMean(M1),calcMean(M2)
print(X1)
print(X2)

[[168.90525]
 [ 14.04775]
 [ 20.8325 ]]
[[45.7906]
 [ 4.778 ]
 [10.758 ]]


### Получим ковариационные матрицы для М1 и М2

In [9]:
def getCovMatr(M, vec):
    s = np.zeros((3,3))
    for row in M:
        s += (np.reshape(row,(-1,1)) - vec).dot((np.reshape(row,(-1,1)) - vec).T)
    return s/len(M)  

S1,S2 = getCovMatr(M1,X1),getCovMatr(M2,X2)
print(S1)
print(S2)

[[1023.94899519   55.61976531   28.91242912]
 [  55.61976531    5.64686869   10.27363737]
 [  28.91242912   10.27363737   44.87966475]]
[[145.84115864  -6.6084026   22.784783  ]
 [ -6.6084026    0.371782    -0.9024838 ]
 [ 22.784783    -0.9024838    5.750306  ]]


### Получим обьединенную ковариационную матрицу

In [11]:
def getOCV(matrs,args):
    constant = 1/(np.sum(args) - len(matrs))
    summ, index = 0, 0
    for row in matrs:
        summ +=args[index]*row
        index+=1
    return constant*summ
        
ocv = getOCV([S1,S2],[4,5])
print(ocv)

[[689.28596771  27.06243546  32.79623307]
 [ 27.06243546   3.49234068   5.22601864]
 [ 32.79623307   5.22601864  29.75288414]]


### Получим обратную обьединенную ковариационную матрицу

In [12]:
ocvInv = np.linalg.inv(ocv)
print(ocvInv)

[[ 0.00209699 -0.01735154  0.00073626]
 [-0.01735154  0.53201446 -0.07432061]
 [ 0.00073626 -0.07432061  0.04585284]]


### Вектор дискриминантных множителей

In [13]:
A = np.dot(ocvInv,np.subtract(X1,X2))
print(A)

[[ 0.1047437 ]
 [ 2.04666965]
 [-0.13634417]]


### Значения дискриминантной функции:

In [14]:
def getFuncValues(A,M):
    vec = []
    for row in M:
        vec.append(np.dot(row,A))
    return np.array(vec)

F1 = getFuncValues(A,M1)
F2 = getFuncValues(A,M2)
print(F1)
print(F2)

[[55.38189512]
 [43.47767643]
 [39.41122432]
 [36.13909921]]
[[12.44346162]
 [13.48770114]
 [12.47078605]
 [13.57388272]
 [13.56653676]]


### Средние значения F для каждой из групп:

In [15]:
Fm = np.array([np.mean(F1), np.mean(F2)])
print(Fm)

[43.60247377 13.10847366]


### Константа (общее среднее) дискриминации:

In [16]:
Fms = np.mean(Fm) 
print(Fms)

28.355473714207854


## Распределение (дискриминация) объектов из M0 по обучающим выборкам M1, M2:

In [17]:
M0 = M0.iloc[:,1:].to_numpy()
F0 = getFuncValues(A,M0)
print(F0)

[[23.68913879]
 [30.11342418]
 [43.47660675]]


In [18]:
def classificate(needle, Fm):
    res, index, Fms = {}, 0, np.mean(Fm)
    for Fi in needle:
        index+=1
        if Fm[0] > Fm[1]:
            res[index] = [Fi[0],'M1'] if (Fi[0] > Fms) else [Fi[0],'M2'] 
        else:
            res[index] = [Fi[0],'M2'] if (Fi[0] > Fms) else [Fi[0],'M1']
    return res

In [19]:
result = classificate(F0, Fm)
result = pd.DataFrame.from_dict(result, orient = "index", columns = ['F','Группа'])
HTML(result.to_html())

,F,Группа
1,23.689139,M2
2,30.113424,M1
3,43.476607,M1
